Caso a resolver: https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0

### Ejemplo 1

In [82]:
# Inicializar variables/parametros

import numpy as np
import pandas as pd
from gekko import GEKKO

m = GEKKO(remote=False)

# Matriz de costos
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])

# Matriz de Demanda (demand)
cust_demands = np.array([35000, 22000, 18000, 30000])

# Matriz de suministro (supply)
warehouse_supply = np.array([60000, 80000])

# Cantidad de elementos
n_customers = len(cust_demands)
n_warehouses = len(warehouse_supply)


# Definir matriz con las variables de decision
X = m.Array(m.Var,(n_warehouses,n_customers))

# Indicar cada uno de los elementos de la matriz X como variable con ciertas condiciones
for i in range(n_warehouses):
    for j in range(n_customers):
        X[i,j] = m.Var(lb=0,integer=1,name="X" + str(i) + "_" + str(j)) # inicializar variables con limites (lb: lower, ub: upper), tipo entero o no y nombre

### NOTAS:
##  m.Var([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.MV([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.Const(value, [name])
##  p = m.Param([value], [name])

# En caso se requiera se puede indicar que una variable sea colocada a valor "constante"
m.fix(X[0,0],val=100)

# Para liberar una variable que se le aplicó "fix" usar m.free()
m.free(X[0,0])

In [77]:
# Agregar restricciones

# restricciones de "Supply"
for i in range(n_warehouses):
    # Generar  y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[i,:]) <= warehouse_supply[i]) # Xi1 + Xi2 + Xi3 ... + Xin <= warehouse_supply(i)
    print(np.sum(X[i,:]) <= warehouse_supply[i])


# restricciones de "Demand"
for j in range(n_customers):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[:,j]) >= cust_demands[j]) # X1j + X2j + X3j ... + Xmj <= customer_demand(j)
    print(np.sum(X[:,j]) >= cust_demands[j])


(((x0_0+x0_1)+x0_2)+x0_3)<=60000
(((x1_0+x1_1)+x1_2)+x1_3)<=80000
(x0_0+x1_0)>=35000
(x0_1+x1_1)>=22000
(x0_2+x1_2)>=18000
(x0_3+x1_3)>=30000


In [78]:
# Calcular la funcion "objetivo" mediante multiplication uno a uno
objetivo = np.sum(X*cost_matrix)
m.Obj(objetivo) # Objetivo por defecto es "minimizar"
print(objetivo)

(((((((((x0_0)*(1.0))+((x0_1)*(3.0)))+((x0_2)*(0.5)))+((x0_3)*(4.0)))+((x1_0)*(2.5)))+((x1_1)*(5.0)))+((x1_2)*(1.5)))+((x1_3)*(2.5)))


In [79]:
# minimizar el objetivo
m.solve(disp=1)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  22
   Intermediates:  0
   Connections  :  3
   Equations    :  7
   Residuals    :  7
 
 Number of state variables:    22
 Number of total equations: -  6
 Number of slack variables: -  6
 ---------------------------------------
 Degrees of freedom       :    10
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
  

In [80]:
# Ver valores obtenidos

# valor de la funcion objetivo
print('Valor de funcion objetivo: ' + str(m.options.objfcnval))

# Valores de la variable de decision
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        print('X' + str(i) + "_" + str(j) + ":",X[i,j].value[0])

Valor de funcion objetivo: 200000.0
X0_0: 35000.0
X0_1: 22000.0
X0_2: 3000.0
X0_3: 0.0
X1_0: 0.0
X1_1: 0.0
X1_2: 15000.0
X1_3: 30000.0


In [81]:
# minimizar el objetivo
try:
    m.solve(disp=1)
except Exception as e:
    print('No se pudo resolver, mensaje:',e)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  22
   Intermediates:  0
   Connections  :  3
   Equations    :  7
   Residuals    :  7
 
 Number of state variables:    22
 Number of total equations: -  6
 Number of slack variables: -  6
 ---------------------------------------
 Degrees of freedom       :    10
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
  

In [8]:
# Ver valores obtenidos

try:
    # valor de la funcion objetivo
    print('Valor de funcion objetivo: ' + str(m.options.objfcnval))

    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            print('X' + str(i) + "_" + str(j) + ":",X[i,j].value[0])
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Valor de funcion objetivo: 200000.0
X0_0: 35000.0
X0_1: 22000.0
X0_2: 3000.0
X0_3: 0.0
X1_0: 0.0
X1_1: 0.0
X1_2: 15000.0
X1_3: 30000.0


### Ejemplo 2

Es una variante al ejemplo 1. El enfoque cambia en que se usa "m.MV" en lugar de "m.Var"

In [9]:
# Inicializar variables/parametros

import numpy as np
import pandas as pd
from gekko import GEKKO

m = GEKKO(remote=False)

# Matriz de costos
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])

# Matriz de Demanda (demand)
cust_demands = np.array([35000, 22000, 18000, 30000])

# Matriz de suministro (supply)
warehouse_supply = np.array([60000, 80000])

# Cantidad de elementos
n_customers = len(cust_demands)
n_warehouses = len(warehouse_supply)


# Definir matriz con las variables de decision
X = m.Array(m.Var,(n_warehouses,n_customers))

# Indicar cada uno de los elementos de la matriz X como variable con ciertas condiciones
for i in range(n_warehouses):
    for j in range(n_customers):
        X[i,j] = m.MV(lb=0,integer=1,name="X" + str(i) + "_" + str(j)) # inicializar variables con limites (lb: lower, ub: upper), tipo entero o no y nombre
        X[i,j].STATUS = 1  # Indicar que se puede tratar como variable (STATUS = 1)

# VITAL: con STATUS = 1 se considera variable (calcular el valor) pero con STATUS = 0 no se calcula nuevo valor, es como si fuera constante

### NOTAS:
##  m.Var([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.MV([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.Const(value, [name])
##  p = m.Param([value], [name])

# En caso se requiera se puede indicar que una variable sea colocada a valor "constante"
m.fix(X[0,0],val=100)

# Para liberar una variable que se le aplicó "fix" usar m.free()
m.free(X[0,0])

In [10]:
# Agregar restricciones

# restricciones de "Supply"
for i in range(n_warehouses):
    # Generar  y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[i,:]) <= warehouse_supply[i]) # Xi1 + Xi2 + Xi3 ... + Xin <= warehouse_supply(i)
    print(np.sum(X[i,:]) <= warehouse_supply[i])


# restricciones de "Demand"
for j in range(n_customers):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[:,j]) >= cust_demands[j]) # X1j + X2j + X3j ... + Xmj <= customer_demand(j)
    print(np.sum(X[:,j]) >= cust_demands[j])


(((int_x0_0+int_x0_1)+int_x0_2)+int_x0_3)<=60000
(((int_x1_0+int_x1_1)+int_x1_2)+int_x1_3)<=80000
(int_x0_0+int_x1_0)>=35000
(int_x0_1+int_x1_1)>=22000
(int_x0_2+int_x1_2)>=18000
(int_x0_3+int_x1_3)>=30000


In [11]:
# Calcular la funcion "objetivo" mediante multiplication uno a uno
objetivo = np.sum(X*cost_matrix)
m.Obj(objetivo) # Objetivo por defecto es "minimizar"
print(objetivo)

(((((((((int_x0_0)*(1.0))+((int_x0_1)*(3.0)))+((int_x0_2)*(0.5)))+((int_x0_3)*(4.0)))+((int_x1_0)*(2.5)))+((int_x1_1)*(5.0)))+((int_x1_2)*(1.5)))+((int_x1_3)*(2.5)))


In [12]:
# minimizar el objetivo
try:
    m.solve(disp=1)
except Exception as e:
    print('No se pudo resolver, mensaje:',e)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  22
   Intermediates:  0
   Connections  :  3
   Equations    :  7
   Residuals    :  7
 
 Number of state variables:    22
 Number of total equations: -  6
 Number of slack variables: -  6
 ---------------------------------------
 Degrees of freedom       :    10
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
  

In [13]:
# Ver valores obtenidos

try:
    # valor de la funcion objetivo
    print('Valor de funcion objetivo: ' + str(m.options.objfcnval))

    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            print('X' + str(i) + "_" + str(j) + ":",X[i,j].value[0])
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Valor de funcion objetivo: 200000.0
X0_0: 35000.0
X0_1: 22000.0
X0_2: 3000.0
X0_3: 0.0
X1_0: 0.0
X1_1: 0.0
X1_2: 15000.0
X1_3: 30000.0


### Ejemplo 3

Ejemplo con restriccion de algunas m.Var a valores especificos

In [14]:
# Inicializar variables/parametros

import numpy as np
import pandas as pd
from gekko import GEKKO

m = GEKKO(remote=False)

# Matriz de costos
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])

# Matriz de Demanda (demand)
cust_demands = np.array([35000, 22000, 18000, 30000])

# Matriz de suministro (supply)
warehouse_supply = np.array([60000, 80000])

# Cantidad de elementos
n_customers = len(cust_demands)
n_warehouses = len(warehouse_supply)


# Definir matriz con las variables de decision
X = m.Array(m.Var,(n_warehouses,n_customers))

# Indicar cada uno de los elementos de la matriz X como variable con ciertas condiciones
for i in range(n_warehouses):
    for j in range(n_customers):
        X[i,j] = m.MV(lb=0,integer=1,name="X" + str(i) + "_" + str(j)) # inicializar variables con limites (lb: lower, ub: upper), tipo entero o no y nombre
        X[i,j].STATUS = 1  # Indicar que se puede tratar como variable (STATUS = 1)

# VITAL: con STATUS = 1 se considera variable (calcular el valor) pero con STATUS = 0 no se calcula nuevo valor, es como si fuera constante

### NOTAS:
##  m.Var([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.MV([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.Const(value, [name])
##  p = m.Param([value], [name])

# En caso se requiera se puede indicar que una variable sea colocada a valor "constante"
m.fix(X[0,0],val=200)
m.fix(X[0,1],val=200)

# Para liberar una variable que se le aplicó "fix" usar m.free()
#m.free(X[0,0])

In [15]:
# Agregar restricciones

# restricciones de "Supply"
for i in range(n_warehouses):
    # Generar  y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[i,:]) <= warehouse_supply[i]) # Xi1 + Xi2 + Xi3 ... + Xin <= warehouse_supply(i)
    print(np.sum(X[i,:]) <= warehouse_supply[i])


# restricciones de "Demand"
for j in range(n_customers):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[:,j]) >= cust_demands[j]) # X1j + X2j + X3j ... + Xmj <= customer_demand(j)
    print(np.sum(X[:,j]) >= cust_demands[j])


(((int_x0_0+int_x0_1)+int_x0_2)+int_x0_3)<=60000
(((int_x1_0+int_x1_1)+int_x1_2)+int_x1_3)<=80000
(int_x0_0+int_x1_0)>=35000
(int_x0_1+int_x1_1)>=22000
(int_x0_2+int_x1_2)>=18000
(int_x0_3+int_x1_3)>=30000


In [16]:
# Calcular la funcion "objetivo" mediante multiplication uno a uno
objetivo = np.sum(X*cost_matrix)
m.Obj(objetivo) # Objetivo por defecto es "minimizar"
print(objetivo)

(((((((((int_x0_0)*(1.0))+((int_x0_1)*(3.0)))+((int_x0_2)*(0.5)))+((int_x0_3)*(4.0)))+((int_x1_0)*(2.5)))+((int_x1_1)*(5.0)))+((int_x1_2)*(1.5)))+((int_x1_3)*(2.5)))


In [17]:
# minimizar el objetivo
try:
    m.solve(disp=1)
except Exception as e:
    print('No se pudo resolver, mensaje:',e)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  22
   Intermediates:  0
   Connections  :  4
   Equations    :  7
   Residuals    :  7
 
 Number of state variables:    20
 Number of total equations: -  6
 Number of slack variables: -  6
 ---------------------------------------
 Degrees of freedom       :    8
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
   

In [18]:
# Ver valores obtenidos

try:
    # valor de la funcion objetivo
    print('Valor de funcion objetivo: ' + str(m.options.objfcnval))

    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            print('X' + str(i) + "_" + str(j) + ":",X[i,j].value[0])
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Valor de funcion objetivo: 290700.0
X0_0: 200.0
X0_1: 200.0
X0_2: 18000.0
X0_3: 6600.0
X1_0: 34800.0
X1_1: 21800.0
X1_2: 0.0
X1_3: 23400.0


Se observa la variable "X0_0" y "X0_1" mantiene el valor constante (fix) seteado al inicio

### Ejemplo 4

Ejemplo de no solucion debido a condiciones impuestas (fix)

In [20]:
# Inicializar variables/parametros

import numpy as np
import pandas as pd
from gekko import GEKKO

m = GEKKO(remote=False)

# Matriz de costos
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])

# Matriz de Demanda (demand)
cust_demands = np.array([35000, 22000, 18000, 30000])

# Matriz de suministro (supply)
warehouse_supply = np.array([60000, 80000])

# Cantidad de elementos
n_customers = len(cust_demands)
n_warehouses = len(warehouse_supply)


# Definir matriz con las variables de decision
X = m.Array(m.Var,(n_warehouses,n_customers))

# Indicar cada uno de los elementos de la matriz X como variable con ciertas condiciones
for i in range(n_warehouses):
    for j in range(n_customers):
        X[i,j] = m.MV(lb=0,integer=1,name="X" + str(i) + "_" + str(j)) # inicializar variables con limites (lb: lower, ub: upper), tipo entero o no y nombre
        X[i,j].STATUS = 1  # Indicar que se puede tratar como variable (STATUS = 1)

# VITAL: con STATUS = 1 se considera variable (calcular el valor) pero con STATUS = 0 no se calcula nuevo valor, es como si fuera constante

### NOTAS:
##  m.Var([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.MV([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.Const(value, [name])
##  p = m.Param([value], [name])

# En caso se requiera se puede indicar que una variable sea colocada a valor "constante"
m.fix(X[0,0],val=100)
m.fix(X[1,0],val=100)

# Para liberar una variable que se le aplicó "fix" usar m.free()
#m.free(X[0,0])

In [21]:
# Agregar restricciones

# restricciones de "Supply"
for i in range(n_warehouses):
    # Generar  y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[i,:]) <= warehouse_supply[i]) # Xi1 + Xi2 + Xi3 ... + Xin <= warehouse_supply(i)
    print(np.sum(X[i,:]) <= warehouse_supply[i])


# restricciones de "Demand"
for j in range(n_customers):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[:,j]) >= cust_demands[j]) # X1j + X2j + X3j ... + Xmj <= customer_demand(j)
    print(np.sum(X[:,j]) >= cust_demands[j])


(((int_x0_0+int_x0_1)+int_x0_2)+int_x0_3)<=60000
(((int_x1_0+int_x1_1)+int_x1_2)+int_x1_3)<=80000
(int_x0_0+int_x1_0)>=35000
(int_x0_1+int_x1_1)>=22000
(int_x0_2+int_x1_2)>=18000
(int_x0_3+int_x1_3)>=30000


In [22]:
# Calcular la funcion "objetivo" mediante multiplication uno a uno
objetivo = np.sum(X*cost_matrix)
m.Obj(objetivo) # Objetivo por defecto es "minimizar"
print(objetivo)

(((((((((int_x0_0)*(1.0))+((int_x0_1)*(3.0)))+((int_x0_2)*(0.5)))+((int_x0_3)*(4.0)))+((int_x1_0)*(2.5)))+((int_x1_1)*(5.0)))+((int_x1_2)*(1.5)))+((int_x1_3)*(2.5)))


In [23]:
# minimizar el objetivo
try:
    m.solve(disp=1)
except Exception as e:
    print('No se pudo resolver, mensaje:',e)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  22
   Intermediates:  0
   Connections  :  4
   Equations    :  7
   Residuals    :  7
 
 Number of state variables:    20
 Number of total equations: -  6
 Number of slack variables: -  6
 ---------------------------------------
 Degrees of freedom       :    8
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
   

In [24]:
# Ver valores obtenidos

try:
    # valor de la funcion objetivo
    print('Valor de funcion objetivo: ' + str(m.options.objfcnval))

    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            print('X' + str(i) + "_" + str(j) + ":",X[i,j].value[0])
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Valor de funcion objetivo: 0.0
No se pudo encontrar la solucion


### Ejemplo 5

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html#scipy.optimize.linear_sum_assignment

Ejemplo base a resolver: https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [113]:
# Inicializar variables/parametros

import numpy as np
import pandas as pd
from gekko import GEKKO

m = GEKKO(remote=False)

# Matriz de tiempos - NOTA: Se agregaron mas registros de personas a la tabla de tiempos
costos = np.array([[43.5,47.1,48.4,38.2],
                    [45.5,42.1,49.6,36.8],
                    [43.4,39.1,42.1,43.2],
                    [46.5,44.1,44.5,41.2],
                    [46.3,49.8,50.4,39.4],
                    [48.7,49.6,49.1,35.2],
                    [48.9,49.8,49.5,35.2],
                    [44.1,48.8,50.4,37.2],
                    [47.9,48.3,45.7,39.8],
                    [48.9,46.3,43.5,38.6]])

estudiantes = ["Mario","Marcus","Luis","Lisandro","Pepe","Bruno","Ramiro","Christian","Pablo","Sandro"]
estilos = ["backstroke","breatsroke","butterfly","freestyle"]

# NOTA: len(estudiantes) == costos.shape[0] y len(estilos) == costos.shape[1]


# Definir matriz con las variables de decision
#X = m.Array(m.Var,(len(estudiantes),len(estilos)))
X = m.Array(m.MV,(len(estudiantes),len(estilos)),lb=0,ub=1,integer=True)

# Indicar cada uno de los elementos de la matriz X como variable con ciertas condiciones
for i in range(len(estudiantes)):
    for j in range(len(estilos)):
        # Las variables son enteros y binarios (valores {0,1})
        #X[i,j] = m.MV(lb=0,ub=1,integer=1,name= str(estudiantes[i]) + "_" + str(estilos[j])) # inicializar variables con limites (lb: lower, ub: upper), tipo entero o no y nombre
        X[i,j].STATUS = 1  # Indicar que se puede tratar como variable (STATUS = 1)

X = np.array(X)

# VITAL: con STATUS = 1 se considera variable (calcular el valor) pero con STATUS = 0 no se calcula nuevo valor, es como si fuera constante

### NOTAS:
##  m.Var([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.MV([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.Const(value, [name])
##  p = m.Param([value], [name])

# En caso se requiera se puede indicar que una variable sea colocada a valor "constante"
#m.fix(X[0,0],val=100)

# Para liberar una variable que se le aplicó "fix" usar m.free()
#m.free(X[0,0])

print("valores iniciales matriz X:\n",X)

valores iniciales matriz X:
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [114]:
# Agregar restricciones

# Restricciones en cada fila :: maximo 1 prueba por persona maximo
for i in range(len(estudiantes)):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[i,:]) <= 1) # maximo 1 estilo por persona
    print(np.sum(X[i,:]) <= 1)


# Restricciones en cada columna :: solo 1 persona por estilo
for j in range(len(estilos)):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[:,j]) == 1) # solo 1 persona por estilo
    print(np.sum(X[:,j]) == 1)

(((int_p1+int_p2)+int_p3)+int_p4)<=1
(((int_p5+int_p6)+int_p7)+int_p8)<=1
(((int_p9+int_p10)+int_p11)+int_p12)<=1
(((int_p13+int_p14)+int_p15)+int_p16)<=1
(((int_p17+int_p18)+int_p19)+int_p20)<=1
(((int_p21+int_p22)+int_p23)+int_p24)<=1
(((int_p25+int_p26)+int_p27)+int_p28)<=1
(((int_p29+int_p30)+int_p31)+int_p32)<=1
(((int_p33+int_p34)+int_p35)+int_p36)<=1
(((int_p37+int_p38)+int_p39)+int_p40)<=1
(((((((((int_p1+int_p5)+int_p9)+int_p13)+int_p17)+int_p21)+int_p25)+int_p29)+int_p33)+int_p37)=1
(((((((((int_p2+int_p6)+int_p10)+int_p14)+int_p18)+int_p22)+int_p26)+int_p30)+int_p34)+int_p38)=1
(((((((((int_p3+int_p7)+int_p11)+int_p15)+int_p19)+int_p23)+int_p27)+int_p31)+int_p35)+int_p39)=1
(((((((((int_p4+int_p8)+int_p12)+int_p16)+int_p20)+int_p24)+int_p28)+int_p32)+int_p36)+int_p40)=1


In [115]:
# Calcular la funcion "objetivo" mediante multiplication uno a uno
objetivo = np.sum(X*costos) 
m.Obj(objetivo) # Objetivo por defecto es "minimizar"
print(objetivo)

(((((((((((((((((((((((((((((((((((((((((int_p1)*(43.5))+((int_p2)*(47.1)))+((int_p3)*(48.4)))+((int_p4)*(38.2)))+((int_p5)*(45.5)))+((int_p6)*(42.1)))+((int_p7)*(49.6)))+((int_p8)*(36.8)))+((int_p9)*(43.4)))+((int_p10)*(39.1)))+((int_p11)*(42.1)))+((int_p12)*(43.2)))+((int_p13)*(46.5)))+((int_p14)*(44.1)))+((int_p15)*(44.5)))+((int_p16)*(41.2)))+((int_p17)*(46.3)))+((int_p18)*(49.8)))+((int_p19)*(50.4)))+((int_p20)*(39.4)))+((int_p21)*(48.7)))+((int_p22)*(49.6)))+((int_p23)*(49.1)))+((int_p24)*(35.2)))+((int_p25)*(48.9)))+((int_p26)*(49.8)))+((int_p27)*(49.5)))+((int_p28)*(35.2)))+((int_p29)*(44.1)))+((int_p30)*(48.8)))+((int_p31)*(50.4)))+((int_p32)*(37.2)))+((int_p33)*(47.9)))+((int_p34)*(48.3)))+((int_p35)*(45.7)))+((int_p36)*(39.8)))+((int_p37)*(48.9)))+((int_p38)*(46.3)))+((int_p39)*(43.5)))+((int_p40)*(38.6)))


In [116]:
# minimizar el objetivo
try:
    m.solve(disp=1)
except Exception as e:
    print('No se pudo resolver, mensaje:',e)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  50
   Intermediates:  0
   Connections  :  0
   Equations    :  15
   Residuals    :  15
 
 Number of state variables:    50
 Number of total equations: -  14
 Number of slack variables: -  10
 ---------------------------------------
 Degrees of freedom       :    26
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL)

In [117]:
# Ver valores obtenidos

try:
    # valor de la funcion objetivo
    print('Tiempo total: ' + str(m.options.objfcnval))

    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            print(str(estudiantes[i]) + "_" + str(estilos[j]) + ":",[np.round(ii,5) for ii in X[i,j].value])
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Tiempo total: 161.30000352
Mario_backstroke: [1.0]
Mario_breatsroke: [0.0]
Mario_butterfly: [0.0]
Mario_freestyle: [0.0]
Marcus_backstroke: [0.0]
Marcus_breatsroke: [0.0]
Marcus_butterfly: [0.0]
Marcus_freestyle: [0.0]
Luis_backstroke: [0.0]
Luis_breatsroke: [1.0]
Luis_butterfly: [0.0]
Luis_freestyle: [0.0]
Lisandro_backstroke: [0.0]
Lisandro_breatsroke: [0.0]
Lisandro_butterfly: [0.0]
Lisandro_freestyle: [0.0]
Pepe_backstroke: [0.0]
Pepe_breatsroke: [0.0]
Pepe_butterfly: [0.0]
Pepe_freestyle: [0.0]
Bruno_backstroke: [0.0]
Bruno_breatsroke: [0.0]
Bruno_butterfly: [0.0]
Bruno_freestyle: [0.49374]
Ramiro_backstroke: [0.0]
Ramiro_breatsroke: [0.0]
Ramiro_butterfly: [0.0]
Ramiro_freestyle: [0.50626]
Christian_backstroke: [0.0]
Christian_breatsroke: [0.0]
Christian_butterfly: [0.0]
Christian_freestyle: [0.0]
Pablo_backstroke: [0.0]
Pablo_breatsroke: [0.0]
Pablo_butterfly: [0.0]
Pablo_freestyle: [0.0]
Sandro_backstroke: [0.0]
Sandro_breatsroke: [0.0]
Sandro_butterfly: [1.0]
Sandro_freestyle:

In [118]:
# Seleccionar los nadadores

try:
    # valor de la funcion objetivo
    print('Tiempo total: ' + str(m.options.objfcnval))
    
    # diccionario para guardar resultados
    seleccionados = dict()
    
    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # Seleccionar las X con valor mayor a 0 (idealmente valor 1)
            for k in range(len(X[i,j].value)):
                # En caso sea mayor a un umbral minimo, agregar a lista de seleccionados
                val_min = 1e-2
                if(X[i,j].value[k] > val_min):
                    seleccionados[str(estilos[j]) + "_" + str(estudiantes[i])] = [costos[i,j],]
            
    # Mostrar por pantalla
    print(seleccionados)
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Tiempo total: 161.30000352
{'backstroke_Mario': [43.5], 'breatsroke_Luis': [39.1], 'freestyle_Bruno': [35.2], 'freestyle_Ramiro': [35.2], 'butterfly_Sandro': [43.5]}


### Ejemplo 6

Similar a ejemplo 5 pero con caso de datos vacios

Ejemplo base a resolver: https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [38]:
# Inicializar variables/parametros

import numpy as np
import pandas as pd
from gekko import GEKKO

m = GEKKO(remote=False)

# Matriz de tiempos - NOTA: Se agregaron mas registros de personas a la tabla de tiempos
costos = np.array([[43.5,None,48.4,38.2],
                    [45.5,49.9,None,36.8],
                    [43.4,None,42.1,43.2],
                    [46.3,None,50.4,39.4],
                    [48.7,53.6,49.1,None],
                    [None,57.8,49.5,None],
                    [47.9,None,45.7,39.8],
                    [48.9,50.3,43.5,None]])

# Las celdas sin valor combiar por valores extremos de forma que no afecten el objetivo de minimizar (posterior)
valor_colocar = 1000000
costos = np.where(costos == None, valor_colocar, costos)
print("matriz costos:\n",costos)

# Nombre de personas y estilos segun la matriz
estudiantes = ["Mario","Marcus","Luis","Lisandro","Pepe","Bruno","Ramiro","Christian"]
estilos = ["backstroke","breatsroke","butterfly","freestyle"]

# NOTA: len(estudiantes) == costos.shape[0] y len(estilos) == costos.shape[1]


# Definir matriz con las variables de decision
#X = m.Array(m.Var,(len(estudiantes),len(estilos)))
X = m.Array(m.MV,(len(estudiantes),len(estilos)),lb=0,ub=1,integer=True)

# Indicar cada uno de los elementos de la matriz X como variable con ciertas condiciones
for i in range(len(estudiantes)):
    for j in range(len(estilos)):
        # Las variables son enteros y binarios (valores {0,1})
        #X[i,j] = m.MV(lb=0,ub=1,integer=1,name= str(estudiantes[i]) + "_" + str(estilos[j])) # inicializar variables con limites (lb: lower, ub: upper), tipo entero o no y nombre
        X[i,j].STATUS = 1  # Indicar que se puede tratar como variable (STATUS = 1)

X = np.array(X)

# VITAL: con STATUS = 1 se considera variable (calcular el valor) pero con STATUS = 0 no se calcula nuevo valor, es como si fuera constante

### NOTAS:
##  m.Var([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.MV([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.Const(value, [name])
##  p = m.Param([value], [name])

# En caso se requiera se puede indicar que una variable sea colocada a valor "constante"
#m.fix(X[0,0],val=100)

# Para liberar una variable que se le aplicó "fix" usar m.free()
#m.free(X[0,0])

print("\nvalores iniciales matriz X:\n",X)

matriz costos:
 [[43.5 1000000 48.4 38.2]
 [45.5 49.9 1000000 36.8]
 [43.4 1000000 42.1 43.2]
 [46.3 1000000 50.4 39.4]
 [48.7 53.6 49.1 1000000]
 [1000000 57.8 49.5 1000000]
 [47.9 1000000 45.7 39.8]
 [48.9 50.3 43.5 1000000]]

valores iniciales matriz X:
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [39]:
# Agregar restricciones

# Restricciones en cada fila :: maximo 1 prueba por persona maximo
for i in range(len(estudiantes)):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[i,:]) <= 1) # maximo 1 estilo por persona
    print(np.sum(X[i,:]) <= 1)


# Restricciones en cada columna :: solo 1 persona por estilo
for j in range(len(estilos)):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[:,j]) == 1) # solo 1 persona por estilo
    print(np.sum(X[:,j]) == 1)

(((int_p1+int_p2)+int_p3)+int_p4)<=1
(((int_p5+int_p6)+int_p7)+int_p8)<=1
(((int_p9+int_p10)+int_p11)+int_p12)<=1
(((int_p13+int_p14)+int_p15)+int_p16)<=1
(((int_p17+int_p18)+int_p19)+int_p20)<=1
(((int_p21+int_p22)+int_p23)+int_p24)<=1
(((int_p25+int_p26)+int_p27)+int_p28)<=1
(((int_p29+int_p30)+int_p31)+int_p32)<=1
(((((((int_p1+int_p5)+int_p9)+int_p13)+int_p17)+int_p21)+int_p25)+int_p29)=1
(((((((int_p2+int_p6)+int_p10)+int_p14)+int_p18)+int_p22)+int_p26)+int_p30)=1
(((((((int_p3+int_p7)+int_p11)+int_p15)+int_p19)+int_p23)+int_p27)+int_p31)=1
(((((((int_p4+int_p8)+int_p12)+int_p16)+int_p20)+int_p24)+int_p28)+int_p32)=1


In [40]:
# Calcular la funcion "objetivo" mediante multiplication uno a uno
objetivo = np.sum(X*costos) 
m.Obj(objetivo) # Objetivo por defecto es "minimizar"
print(objetivo)

(((((((((((((((((((((((((((((((((int_p1)*(43.5))+((int_p2)*(1000000)))+((int_p3)*(48.4)))+((int_p4)*(38.2)))+((int_p5)*(45.5)))+((int_p6)*(49.9)))+((int_p7)*(1000000)))+((int_p8)*(36.8)))+((int_p9)*(43.4)))+((int_p10)*(1000000)))+((int_p11)*(42.1)))+((int_p12)*(43.2)))+((int_p13)*(46.3)))+((int_p14)*(1000000)))+((int_p15)*(50.4)))+((int_p16)*(39.4)))+((int_p17)*(48.7)))+((int_p18)*(53.6)))+((int_p19)*(49.1)))+((int_p20)*(1000000)))+((int_p21)*(1000000)))+((int_p22)*(57.8)))+((int_p23)*(49.5)))+((int_p24)*(1000000)))+((int_p25)*(47.9)))+((int_p26)*(1000000)))+((int_p27)*(45.7)))+((int_p28)*(39.8)))+((int_p29)*(48.9)))+((int_p30)*(50.3)))+((int_p31)*(43.5)))+((int_p32)*(1000000)))


In [41]:
# minimizar el objetivo
try:
    m.solve(disp=1)
except Exception as e:
    print('No se pudo resolver, mensaje:',e)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  40
   Intermediates:  0
   Connections  :  0
   Equations    :  13
   Residuals    :  13
 
 Number of state variables:    40
 Number of total equations: -  12
 Number of slack variables: -  8
 ---------------------------------------
 Degrees of freedom       :    20
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).

In [42]:
# Ver valores obtenidos

try:
    # valor de la funcion objetivo
    print('Tiempo total: ' + str(m.options.objfcnval))

    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            print(str(estudiantes[i]) + "_" + str(estilos[j]) + ":",[np.round(ii,5) for ii in X[i,j].value])
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Tiempo total: 172.70000629
Mario_backstroke: [1.0]
Mario_breatsroke: [0.0]
Mario_butterfly: [0.0]
Mario_freestyle: [0.0]
Marcus_backstroke: [0.0]
Marcus_breatsroke: [0.0]
Marcus_butterfly: [0.0]
Marcus_freestyle: [1.0]
Luis_backstroke: [0.0]
Luis_breatsroke: [0.0]
Luis_butterfly: [1.0]
Luis_freestyle: [0.0]
Lisandro_backstroke: [0.0]
Lisandro_breatsroke: [0.0]
Lisandro_butterfly: [0.0]
Lisandro_freestyle: [0.0]
Pepe_backstroke: [0.0]
Pepe_breatsroke: [0.0]
Pepe_butterfly: [0.0]
Pepe_freestyle: [0.0]
Bruno_backstroke: [0.0]
Bruno_breatsroke: [0.0]
Bruno_butterfly: [0.0]
Bruno_freestyle: [0.0]
Ramiro_backstroke: [0.0]
Ramiro_breatsroke: [0.0]
Ramiro_butterfly: [0.0]
Ramiro_freestyle: [0.0]
Christian_backstroke: [0.0]
Christian_breatsroke: [1.0]
Christian_butterfly: [0.0]
Christian_freestyle: [0.0]


In [44]:
# Seleccionar los nadadores

try:
    # valor de la funcion objetivo
    print('Tiempo total: ' + str(m.options.objfcnval))
    
    # diccionario para guardar resultados
    seleccionados = dict()
    
    # Valores de la variable de decision
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # Seleccionar las X con valor mayor a 0 (idealmente valor 1)
            for k in range(len(X[i,j].value)):
                # En caso sea mayor a un umbral minimo, agregar a lista de seleccionados
                val_min = 1e-2
                if(X[i,j].value[k] > val_min):
                    seleccionados[str(estilos[j]) + "_" + str(estudiantes[i])] = [costos[i,j],]
            
    # Mostrar por pantalla
    print(seleccionados)
            
except Exception as e:
    print('No se pudo encontrar la solucion')

Tiempo total: 172.70000629
{'backstroke_Mario': [43.5], 'freestyle_Marcus': [36.8], 'butterfly_Luis': [42.1], 'breatsroke_Christian': [50.3]}


### Ejemplo 7

In [119]:
# Inicializar variables/parametros

import numpy as np
import pandas as pd
from gekko import GEKKO

m = GEKKO(remote=False)

# Matriz de costos
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])

# Matriz de Demanda (demand)
cust_demands = np.array([3512.5, 22024.7, 18001.9, 30004.4])

# Matriz de suministro (supply)
warehouse_supply = np.array([60000, 80000])

# Cantidad de elementos
n_customers = len(cust_demands)
n_warehouses = len(warehouse_supply)


# Definir matriz con las variables de decision
#X = m.Array(m.Var,(n_warehouses,n_customers))
X = m.Array(m.Var,(n_warehouses,n_customers),lb=0,integer=True)

# Indicar cada uno de los elementos de la matriz X como variable con ciertas condiciones
for i in range(n_warehouses):
    for j in range(n_customers):
        X[i,j] = m.Var(lb=0,integer=1,name="X" + str(i) + "_" + str(j)) # inicializar variables con limites (lb: lower, ub: upper), tipo entero o no y nombre
        
### NOTAS:
##  m.Var([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.MV([value], [lb], [ub], [integer=False], [fixed_initial=True], [name])
##  m.Const(value, [name])
##  p = m.Param([value], [name])

# En caso se requiera se puede indicar que una variable sea colocada a valor "constante"
m.fix(X[0,0],val=100)

# Para liberar una variable que se le aplicó "fix" usar m.free()
m.free(X[0,0])

In [120]:
# Agregar restricciones

# restricciones de "Supply"
for i in range(n_warehouses):
    # Generar  y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[i,:]) <= warehouse_supply[i]) # Xi1 + Xi2 + Xi3 ... + Xin <= warehouse_supply(i)
    print(np.sum(X[i,:]) <= warehouse_supply[i])


# restricciones de "Demand"
for j in range(n_customers):
    # Generar y agregar la restriccion al modelo (Expresion matematica; nombre de restriccion)
    m.Equation(np.sum(X[:,j]) >= cust_demands[j]) # X1j + X2j + X3j ... + Xmj <= customer_demand(j)
    print(np.sum(X[:,j]) >= cust_demands[j])


(((int_x0_0+int_x0_1)+int_x0_2)+int_x0_3)<=60000
(((int_x1_0+int_x1_1)+int_x1_2)+int_x1_3)<=80000
(int_x0_0+int_x1_0)>=3512.5
(int_x0_1+int_x1_1)>=22024.7
(int_x0_2+int_x1_2)>=18001.9
(int_x0_3+int_x1_3)>=30004.4


In [121]:
# Calcular la funcion "objetivo" mediante multiplication uno a uno
objetivo = np.sum(X*cost_matrix)
m.Obj(objetivo) # Objetivo por defecto es "minimizar"
print(objetivo)

(((((((((int_x0_0)*(1.0))+((int_x0_1)*(3.0)))+((int_x0_2)*(0.5)))+((int_x0_3)*(4.0)))+((int_x1_0)*(2.5)))+((int_x1_1)*(5.0)))+((int_x1_2)*(1.5)))+((int_x1_3)*(2.5)))


In [122]:
# minimizar el objetivo
m.solve(disp=1)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  22
   Intermediates:  0
   Connections  :  3
   Equations    :  7
   Residuals    :  7
 
 Number of state variables:    22
 Number of total equations: -  6
 Number of slack variables: -  6
 ---------------------------------------
 Degrees of freedom       :    10
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
  

In [123]:
# Ver valores obtenidos

# valor de la funcion objetivo
print('Valor de funcion objetivo: ' + str(m.options.objfcnval))

# Valores de la variable de decision
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        print('X' + str(i) + "_" + str(j) + ":",[np.round(ii,3) for ii in X[i,j].value])

Valor de funcion objetivo: 153598.55
X0_0: [3512.5]
X0_1: [22024.7]
X0_2: [18001.9]
X0_3: [0.0]
X1_0: [0.0]
X1_1: [0.0]
X1_2: [0.0]
X1_3: [30004.4]
